In [1]:
import os
from skimage import io, transform
import numpy as np
from tqdm import tqdm
from model import FireNet
from data_augmentation import prepare_dataset

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from torch.utils.tensorboard import SummaryWriter

In [ ]:
prepare_dataset()

DOWNLOADING TRAINING DATASET

Downloading: 0% [0 / 510779994] bytes
Downloading: 0% [8192 / 510779994] bytes
Downloading: 0% [16384 / 510779994] bytes
Downloading: 0% [24576 / 510779994] bytes
Downloading: 0% [32768 / 510779994] bytes
Downloading: 0% [40960 / 510779994] bytes
Downloading: 0% [49152 / 510779994] bytes
Downloading: 0% [57344 / 510779994] bytes
Downloading: 0% [65536 / 510779994] bytes
Downloading: 0% [73728 / 510779994] bytes
Downloading: 0% [81920 / 510779994] bytes
Downloading: 0% [90112 / 510779994] bytes
Downloading: 0% [98304 / 510779994] bytes
Downloading: 0% [106496 / 510779994] bytes
Downloading: 0% [114688 / 510779994] bytes
Downloading: 0% [122880 / 510779994] bytes
Downloading: 0% [131072 / 510779994] bytes
Downloading: 0% [139264 / 510779994] bytes
Downloading: 0% [147456 / 510779994] bytes
Downloading: 0% [155648 / 510779994] bytes
Downloading: 0% [163840 / 510779994] bytes
Downloading: 0% [172032 / 510779994] bytes
Downloading: 0% [180224 / 510779994] bytes

In [2]:
writer = SummaryWriter('runs/firenet_experiment_1')

In [3]:
print(torch.cuda.is_available())
net = FireNet()
device = "cuda" if torch.cuda.is_available() else "cpu"
net.to(device)

True


FireNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (drop1): Dropout(p=0.5, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=2304, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=2, bias=True)
)

In [4]:
TRAINING_PATH='training_dataset'
CATEGORIES = ['Fire', 'NoFire']

In [5]:
class TrainingSet(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        classes = []
        images = []
        for category in CATEGORIES:
            path = os.path.join(TRAINING_PATH, category)
            class_num = CATEGORIES.index(category)
            
            for img in tqdm(os.listdir(path)):
                try:
                    image = io.imread(os.path.join(path, img))
                    if (image.shape[2] == 3):
                        if (self.transform is not None):
                            image = self.transform(image)
                            images.append(image)
                            classes.append(class_num)
                except Exception as e:
                    pass
        self.set = {'image': images, 'class': classes}
    def __len__(self):
        return len(self.set['class'])
    def __getitem__(self, idx):
        image = self.set['image'][idx]
        classe = self.set['class'][idx]
        sample = {'image': image, 'class': classe}
        return sample
            
        

In [6]:
training_set= TrainingSet(transform=transforms.ToTensor())
print(len(training_set))

100%|██████████| 5172/5172 [00:01<00:00, 2672.82it/s]

9616


In [7]:
class TestingSet(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        classes = []
        images = []
        for category in CATEGORIES:
            path = os.path.join('testing_dataset', category)
            class_num = CATEGORIES.index(category)
            
            for img in tqdm(os.listdir(path)):
                try:
                    image = io.imread(os.path.join(path, img))
                    if (image.shape[2] == 3):
                        if (self.transform):
                            image = self.transform(image)
                            images.append(image)
                            classes.append(class_num)
                except Exception as e:
                    pass
        self.set = {'image': images, 'class': classes}
    def __len__(self):
        return len(self.set['class'])
    def __getitem__(self, idx):
        image = self.set['image'][idx]
        classe = self.set['class'][idx]
        sample = {'image': image, 'class': classe}
        return sample
            
        

In [8]:
test_set = TestingSet(transform=transforms.ToTensor())
print(len(test_set))

100%|██████████| 278/278 [00:00<00:00, 2600.97it/s]

678


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), eps=1e-7)

In [10]:
trainloader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=4)
net = net.float()
net.train()

FireNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (drop1): Dropout(p=0.5, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=2304, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=2, bias=True)
)

In [11]:
len(trainloader)

301

In [12]:
for epoch in range(100):
    global_loss = 0.0
    for data in tqdm(trainloader):
        inputs, labels = data['image'].to(device), data['class'].to(device)

        optimizer.zero_grad()

        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        global_loss += loss.item()
    print('global loss for epoch %d : %.3f' % (epoch + 1, global_loss/ len(trainloader)))
    writer.add_scalar('Training loss 100', global_loss / len(trainloader), epoch + 1)

  0%|          | 0/301 [00:00<?, ?it/s]

global loss for epoch 1 : 0.515


  0%|          | 0/301 [00:00<?, ?it/s]

global loss for epoch 2 : 0.432


  0%|          | 0/301 [00:00<?, ?it/s]

...

global loss for epoch 98 : 0.020


  0%|          | 0/301 [00:00<?, ?it/s]

global loss for epoch 99 : 0.013


100%|██████████| 301/301 [00:01<00:00, 154.50it/s]

global loss for epoch 100 : 0.023


In [13]:
torch.save(net.state_dict(), './trained_weights100.pth')

In [14]:
net = FireNet()
net.float()
net.cuda()
net.load_state_dict(torch.load('./trained_weights100.pth'))
net.eval()

FireNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (drop1): Dropout(p=0.5, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=2304, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=2, bias=True)
)

In [15]:
trainloader = DataLoader(training_set, batch_size=4, shuffle=False, num_workers=0)
net = net.float()
net.eval()

FireNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (drop1): Dropout(p=0.5, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=2304, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=2, bias=True)
)

In [16]:
correct = 0
total = 0
with torch.no_grad():
    print('evaluate accuracy on training set:')
    for data in tqdm(trainloader):
        images, labels = data['image'].to(device), data['class'].to(device)
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
print ('accuracy: %d' % (100*correct/total))

  6%|▌         | 133/2404 [00:00<00:01, 1328.40it/s]

evaluate accuracy on training set:


100%|██████████| 2404/2404 [00:01<00:00, 1342.64it/s]

accuracy: 100


In [17]:
testloader = DataLoader(test_set, batch_size=4, shuffle=False, num_workers=0)
net = net.float()
net.eval()

FireNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (drop1): Dropout(p=0.5, inplace=False)
  (drop2): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=2304, out_features=256, bias=True)
  (dense2): Linear(in_features=256, out_features=128, bias=True)
  (dense3): Linear(in_features=128, out_features=2, bias=True)
)

In [18]:
len(testloader)

170

In [19]:
correct = 0
total = 0
with torch.no_grad():
    print('evaluate accuracy on training set:')
    for data in tqdm(testloader):
        images, labels = data['image'].to(device), data['class'].to(device)
        outputs = net(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
print ('accuracy: %d' % (100*correct/total))

100%|██████████| 170/170 [00:00<00:00, 1256.80it/s]

evaluate accuracy on training set:
accuracy: 81
